In [1]:
import torch
from torch.nn import functional as F
from torch import nn
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import sentencepiece as spm

block_size = 128
batch_size = 32
n_heads = 8
n_layers = 12
d_model = 128+32 * n_heads
dff = d_model * 4
dropout = 0.8
learning_rate = 5e-4
epochs = 1_000
eval_iters = 20

# block_size = 32
# batch_size = 32
# n_heads = 2
# n_layers = 2
# d_model = 32 * n_heads
# dff = d_model * 4
# dropout = 0.6
# learning_rate = 5e-4
# epochs = 2_000
# eval_iters = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True

# tokenizer = Tokenizer(BPE())
# tokenizer.pre_tokenizer = Whitespace()

# trainer = BpeTrainer(special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "\n"], vocab_size=8000)
# tokenizer.train(files=["text_all.txt"], trainer=trainer)
# tokenizer.save("es_tokenizer.json")

In [2]:
# download texts from https://www.gutenberg.org/cache/epub/70219/pg70219.txt from the internet

# !wget https://www.gutenberg.org/cache/epub/70219/pg70219.txt
# !wget https://www.gutenberg.org/files/50430/50430-0.txt
# !wget https://www.gutenberg.org/cache/epub/42727/pg42727.txt
# !wget https://www.gutenberg.org/cache/epub/15725/pg15725.txt
# !wget https://www.gutenberg.org/files/57654/57654-0.txt
# !wget https://www.gutenberg.org/cache/epub/57303/pg57303.txt
# !wget https://www.gutenberg.org/cache/epub/47287/pg47287.txt
# !wget https://www.gutenberg.org/cache/epub/17013/pg17013.txt

# with open('pg70219.txt', 'r') as f:
#     text = f.read()

# with open('50430-0.txt', 'r') as f:
#     text += f.read()

# with open('pg42727.txt', 'r') as f:
#     text += f.read()

# with open('pg15725.txt', 'r') as f:
#     text += f.read()

# with open('57654-0.txt', 'r') as f:
#     text += f.read()

# with open('pg57303.txt', 'r') as f:
#     text += f.read()

# book_list = ['pg70219.txt', '50430-0.txt', 'pg42727.txt', 'pg15725.txt', '57654-0.txt', 'pg57303.txt', 'pg47287.txt', 'pg17013.txt']

# text = ''
# for book in book_list:
#     with open(book, 'r') as f:
#         text += f.read()

# # save text to file
# with open('text_all.txt', 'w') as f:
#     f.write(text)

In [3]:
class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, d_model, block_size, dropout=0.1):

        super().__init__()
        assert d_model % n_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(d_model, d_model, bias=False)
        self.key = nn.Linear(d_model, d_model, bias=False)
        self.value = nn.Linear(d_model, d_model, bias=False)
        self.att_proj = nn.Linear(d_model, d_model, bias=False)
        self.register_buffer('mask', torch.triu(torch.ones(block_size, block_size).bool(), diagonal=1))

    def forward(self, x):

        q = x
        k = x
        v = x
        B,T,_ = x.shape 
        dk = d_model // n_heads

        # linear projections
        q = self.query(q) 
        k = self.key(k) 
        v = self.value(v) 

        # add number of heads
        q = q.view(B,T,n_heads,dk).permute(0,2,1,3)   # B,T,h,dk
        k = k.view(B,T,n_heads,dk).permute(0,2,1,3)  
        v = v.view(B,T,n_heads,dk).permute(0,2,1,3)  
        
        # attention 
        x = q @ k.transpose(-2,-1) # B,h,T,dk @ B,h,dk,T --> B,h,T,T
        x = x * dk ** -0.5 # B,h,T,T
        x = x.masked_fill(self.mask, float('-inf')) # B,h,T,T
        x = F.softmax(x, dim=(-1)) # B,n_h,T,T 
        x = x @ v  # B,h,T,T @ B,T,h,dv --> B,h,T,dv
        B,h,T,dv = x.shape
        x = x.transpose(2,1).contiguous().view(B,T,h*dv) #B,T,C
        out = self.att_proj(x) # B,T,C

        return out
    
class AttentionLayer(nn.Module):
    def __init__(self, n_heads, d_model, block_size, dropout):
        super().__init__()

        self.att = MultiHeadAttention(n_heads, d_model, block_size, dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.att(x)
        return x
    
class FeedForward(nn.Module):
    def __init__(self, d_model, dff, dropout, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.seq = nn.Sequential(
                    nn.Linear(d_model, dff),
                    nn.ReLU(),
                    nn.Dropout(dropout),
                    nn.Linear(dff, d_model)
                    )

    def forward(self, x):
        x = self.seq(x)
        return x
    
class DecoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, block_size, dropout, dff) :
        super().__init__()

        self.att = AttentionLayer(n_heads, d_model,
                                  block_size, dropout)
        
        self.ffw = FeedForward(d_model, dff, dropout)
        self.lnorm1 = nn.LayerNorm(d_model)
        self.lnorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):

        x = x + self.att(self.lnorm1(x))
        x = x + self.ffw(self.lnorm2(x))

        return x

In [4]:
class Model(nn.Module):

    def __init__(self, vocab_size, block_size, dropout, dff, n_layers, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.block_size = block_size
        self.embedding_table = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(block_size, d_model)

        self.decoder = nn.Sequential(*[DecoderLayer(n_heads,
                                                    d_model,
                                                    block_size,
                                                    dropout,
                                                    dff) 
                                                    for _ in range(n_layers)])

        self.out = nn.Linear(d_model, vocab_size)

    def forward(self, x, targets=None):

        embeds = self.embedding_table(x)
        positions = self.pos_embedding(torch.arange(block_size, device=device))
        x = embeds + positions
        x = self.decoder(x)
        logits = self.out(x)

        if targets == None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(input=logits, target=targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        B, T = idx.shape
        if T < self.block_size:
            # pad the input with zeros if it's less than block_size
            idx = F.pad(idx, (0, self.block_size - T))
        for _ in range(max_new_tokens):
            # use only the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
spm.SentencePieceTrainer.train('--input=text_all.txt --model_prefix=m --vocab_size=8000')
sp = spm.SentencePieceProcessor()
sp.load('m.model')
data = open('text_all.txt').read()
data_ids = sp.encode_as_ids(data)

# Get the vocab
vocab = {sp.id_to_piece(id): id for id in range(sp.get_piece_size())}
vocab_size = len(vocab)

# stoi = {c:i for i, c in enumerate(vocab)}
# itos = {i:c for i, c in enumerate(vocab)}

# encode = lambda s: [stoi[c] for c in s]
# decode = lambda i: ''.join([itos[i] for i in i])

data = torch.tensor(data_ids, dtype=torch.long)

n_tr = int(len(data) * 0.9)
n_val = len(data) - n_tr

train = data[:n_tr]
val = data[n_tr+1:]

def make_batches(split):

    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

Xb, Yb = make_batches('train')
m = Model(vocab_size, block_size, dropout, dff, n_layers).to(device)
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
n_params = sum(p.nelement() for p in m.parameters())
scaler = torch.cuda.amp.GradScaler()
print(f'Number of parameters: {n_params}')

@torch.no_grad()
def estimate_loss(m):
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = make_batches(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=text_all.txt --model_prefix=m --vocab_size=8000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: text_all.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
 

Number of parameters: 27476288


In [5]:
for epoch in range(epochs):
    Xb, Yb = make_batches('train')
    
    with torch.cuda.amp.autocast():
        logits, loss = m(Xb, Yb) # B, C

    scaler.scale(loss).backward()
    scaler.unscale_(optimizer)
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)

    Xb, Yb = make_batches('train')
    logits, loss = m(Xb, Yb) # B, C

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 99:
        l = estimate_loss(m)
        print(f"Iteration {epoch}. Training Loss: {l['train']:.3f}. Evaluation Loss: {l['val']:.3f}")

Iteration 99. Training Loss: 5.902. Evaluation Loss: 6.029
Iteration 199. Training Loss: 5.473. Evaluation Loss: 5.661
Iteration 299. Training Loss: 5.148. Evaluation Loss: 5.442
Iteration 399. Training Loss: 4.959. Evaluation Loss: 5.264
Iteration 499. Training Loss: 4.806. Evaluation Loss: 5.157
Iteration 599. Training Loss: 4.669. Evaluation Loss: 5.063
Iteration 699. Training Loss: 4.549. Evaluation Loss: 5.013
Iteration 799. Training Loss: 4.433. Evaluation Loss: 4.932
Iteration 899. Training Loss: 4.317. Evaluation Loss: 4.889
Iteration 999. Training Loss: 4.240. Evaluation Loss: 4.893
Iteration 1099. Training Loss: 4.189. Evaluation Loss: 4.862
Iteration 1199. Training Loss: 4.043. Evaluation Loss: 4.861


KeyboardInterrupt: 

In [7]:
def generate_text(model, sp, seed_text, max_new_tokens):
    seed_token_ids = sp.encode_as_ids(seed_text)
    seed_tensor = torch.tensor(seed_token_ids, device=device).unsqueeze(0)
    generated_token_ids = model.generate(seed_tensor, max_new_tokens).tolist()[0]
    generated_text = sp.decode_ids(generated_token_ids)
    return generated_text

seed_text = "En un lugar del que no logro acordarme"
generated_text = generate_text(m, sp, seed_text, 500)

print(generated_text)

En un lugar del que no logro acordarme ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ μστ y. Emilio es, compañero de Emilio, y me voiga fué á quien causae un gran ruido, mientras á un agrioso una mirada bajo de hombre. A veces de Udenificame. Al cabo de aquélla subió luego en la boca dentro de la frase de Cosaid, más ungivostórico y en la escala. De esta llamada anrocitaría su empresa que hiciera en padre de estas pisé viviente vocablos, y alguno le traje daré cinco mil pentrio. Lené entre dichosos y les parecias no asentir suelto quieto, como quien se hace al lugar por la suya del temor Telamonio y sólo se abanimeriisé en lo baja arruinismos, no tendrías averiguarción e impusible, sin

In [9]:
readable = []
chunk = 100
for i in range(0, len(generated_text), chunk):
    readable.append(generated_text[i:i+chunk])

print('\n'.join(readable))

En un lugar del que no logro acordarme ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇
  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
 ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  
⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ μστ y. E
milio es, compañero de Emilio, y me voiga fué á quien causae un gran ruido, mientras á un agrioso un
a mirada bajo de hombre. A veces de Udenificame. Al cabo de aquélla subió luego en la boca dentro de
 la frase de Cosaid, más ungivostórico y en la escala. De esta llamada anrocitaría su empresa que hi
ciera en padre de estas pisé viviente vocablos, y alguno le traje daré cinco mil pentrio. Lené entre
 dichosos y les parecias no asentir suelto quieto, como quien se hace al lugar por la suya del temor
 Telamonio y sólo se abanimeriisé en lo baja arruinismos, no tendrías averiguarción e impus